In [ ]:
import pandas as pd
train=pd.read_csv('../input/spaceship-titanic/train.csv')
train.head()

In [ ]:
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
test=pd.read_csv('../input/spaceship-titanic/test.csv')
test.head()

In [ ]:
train.nunique().sort_values(ascending=False)

In [ ]:
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)

In [ ]:
train=train.drop(['PassengerId'],axis=1)
train.head()

In [ ]:
test=test.drop(['PassengerId'],axis=1)
test.head()

In [ ]:
train['Destination'].value_counts()

In [ ]:
train['Name'].value_counts()

In [ ]:
train=train.drop(['Name'],axis=1)
train.head()

In [ ]:
test=test.drop(['Name'],axis=1)
test.head()

In [ ]:
train.info()

In [ ]:
train[['deck', 'num','side']] = train['Cabin'].str.split('/', expand=True)
train=train.drop(['Cabin'],axis=1)
train.head()

In [ ]:
test[['deck', 'num','side']] = test['Cabin'].str.split('/', expand=True)
test=test.drop(['Cabin'],axis=1)
test.head()

In [ ]:
train.nunique().sort_values(ascending=False)

In [ ]:
train['deck'].value_counts()

In [ ]:
train['deck'].unique().tolist()

In [ ]:
test['deck'].unique().tolist()

In [ ]:
train['deck']=train['deck'].replace({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7})
train['deck'].value_counts()

In [ ]:
test['deck']=test['deck'].replace({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'T':7})
test['deck'].value_counts()

In [ ]:
train.head()

In [ ]:
train[['Age','RoomService','FoodCourt',
       'ShoppingMall','Spa','VRDeck','deck','num']]=train[['Age','RoomService','FoodCourt',
       'ShoppingMall','Spa','VRDeck','deck','num']].astype('float')
train.info()

In [ ]:
test[['Age','RoomService','FoodCourt',
       'ShoppingMall','Spa','VRDeck','deck','num']]=test[['Age','RoomService','FoodCourt',
       'ShoppingMall','Spa','VRDeck','deck','num']].astype('float')
test.info()

In [ ]:
train[['HomePlanet','CryoSleep','Destination','VIP','side']].nunique().sort_values(ascending=False)

In [ ]:
train=pd.get_dummies(train,prefix_sep='__')
train.head()

In [ ]:
test=pd.get_dummies(test,prefix_sep='__')
test.head()

In [ ]:
# !rm -r kuma_utils
!git clone https://github.com/analokmaus/kuma_utils.git

In [ ]:
import sys
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

In [ ]:
col=train.columns.tolist()
col.remove('Transported')
col

In [ ]:
%%time
lgbm_imtr = LGBMImputer(n_iter=500)

train_iterimp = lgbm_imtr.fit_transform(train[col])
test_iterimp = lgbm_imtr.transform(test[col])

# Create train test imputed dataframe
train_ = pd.DataFrame(train_iterimp, columns=col)
test = pd.DataFrame(test_iterimp, columns=col)

In [ ]:
train_['Transported'] = train['Transported']
train_.head()

In [ ]:
def undummify(df, prefix_sep="__"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [ ]:
train=undummify(train_)
train.head()

In [ ]:
test=undummify(test)
test.head()

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
y = train.pop('Transported')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

In [ ]:
automl.fit(X_train, y_train, task="classification",metric='log_loss',time_budget=300)

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best log_loss on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, automl.predict(X_train)))

In [ ]:
print(classification_report(y_test, automl.predict(X_test)))

In [ ]:
y_pred = automl.predict(test)
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['Transported'])
df.head()

In [ ]:
sol=pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
sol.head()

In [ ]:
sol['Transported']=df['Transported']
sol.head()

In [ ]:
sol.to_csv('./submission.csv',index=False)